### Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

Исследуем таблицы — выведем первые строки.

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books'''
pd.io.sql.read_sql(sql=text(query), con = con).info()
pd.io.sql.read_sql(sql=text(query), con = con).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''SELECT * FROM authors'''
pd.io.sql.read_sql(sql=text(query), con = con).info()
pd.io.sql.read_sql(sql=text(query), con = con).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''SELECT * FROM publishers'''
pd.io.sql.read_sql(sql=text(query), con = con).info()
pd.io.sql.read_sql(sql=text(query), con = con).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = '''SELECT * FROM ratings'''
pd.io.sql.read_sql(sql=text(query), con = con).info()
pd.io.sql.read_sql(sql=text(query), con = con).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = '''SELECT * FROM reviews'''
pd.io.sql.read_sql(sql=text(query), con = con).info()
pd.io.sql.read_sql(sql=text(query), con = con).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Вывод**

По результатам изучения общей информации можно сделать следующие выводы: 
- База данных содержит информацию о 1000 книг.
- Количество строк в столбцах не различается. Значит, в данных нет пропущенных значений. 
- Данные соответствуют описанию. При этом в таблице `books` присутствует некорректный тип данных в столбце `publication_date`.

### Задания
#### Посчитайте, сколько книг вышло после 1 января 2000 года

In [8]:
query1 = '''
SELECT COUNT(*) number_of_books
FROM books
WHERE publication_date::date > '2000-01-01';
'''
pd.io.sql.read_sql(sql=text(query1), con = con)

,number_of_books
0,819


После 1 января 2000 года вышло 819 книг.

#### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [9]:
query2 = '''
WITH review_data AS (SELECT b.book_id,
    b.title,
    COUNT(DISTINCT rev.review_id) number_of_reviews
FROM books b
LEFT JOIN reviews rev ON b.book_id = rev.book_id
GROUP BY b.book_id)

SELECT rd.book_id,
    rd.title,
    rd.number_of_reviews,
    AVG(rat.rating) average_rating
FROM review_data rd
LEFT JOIN ratings rat ON rd.book_id = rat.book_id
GROUP BY rd.book_id, rd.title, rd.number_of_reviews
ORDER BY book_id;
'''
pd.io.sql.read_sql(sql=text(query2), con = con)

,book_id,title,number_of_reviews,average_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


Для каждой книги посчитали количество обзоров и среднюю оценку.

#### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [10]:
query3 = '''
WITH big_books AS (SELECT publisher_id,
    COUNT(book_id) number_of_books
FROM books
WHERE num_pages > 50
GROUP BY publisher_id)

SELECT p.publisher_id,
    p.publisher,
    bb.number_of_books
FROM big_books bb
LEFT JOIN publishers p ON bb.publisher_id = p.publisher_id
ORDER BY bb.number_of_books DESC
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(query3), con = con)

,publisher_id,publisher,number_of_books
0,212,Penguin Books,42


Издательство Penguin Books выпустило 42 книги – это наибольшее число книг толще 50 страниц.

#### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [11]:
query4 = '''
WITH book_avg_rating AS (SELECT book_id, 
    AVG(rating) avg_rating
FROM ratings
GROUP BY book_id
HAVING COUNT(rating) >= 50)

SELECT a.author_id,
    a.author, 
    AVG(bar.avg_rating) author_avg_rating
FROM books b
JOIN authors a ON b.author_id = a.author_id
JOIN book_avg_rating bar ON b.book_id = bar.book_id
GROUP BY a.author_id, a.author
ORDER BY author_avg_rating DESC
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(query4), con = con)

,author_id,author,author_avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844


Если учитывать только книги с 50 и более оценками, то у J.K. Rowling самая высокая средняя оценка книг.

#### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [12]:
query5 = '''
WITH users_48_ratings AS (SELECT username
FROM ratings
GROUP BY username 
HAVING COUNT(rating_id) > 48),

user_review_count AS (SELECT username, 
COUNT(*) AS review_count
FROM reviews
WHERE username IN (SELECT username FROM users_48_ratings)
GROUP BY username)

SELECT AVG(review_count) average_reviews 
FROM user_review_count;
'''
pd.io.sql.read_sql(sql=text(query5), con = con)

,average_reviews
0,24.0


У пользователей, которые поставили больше 48 оценок, в среднем 24 обзора.

### Общий вывод

Был проведен анализ базы данных и получены следующие результаты:
- после 1 января 2000 года вышло 819 книг;
- для каждой книги были посчитаны количество обзоров и средняя оценка;
- издательство Penguin Books выпустило 42 книги – это наибольшее число книг толще 50 страниц;
- если учитывать только книги с 50 и более оценками, то у J.K. Rowling самая высокая средняя оценка книг;
- у пользователей, которые поставили больше 48 оценок, в среднем 24 обзора.

Чтобы стимулировать приобретать подписку на сервис, в ценностном предложении рекомендуем упомянуть о том, что в сервисе представлена большая подборка книг издательства Penguin Books, а также книги J.K. Rowling, у которой самая высокая средняя оценка книг. Также необходимо привлекать пользователей к написанию обзоров на книги, так как количество обзоров уступает количеству оценок.